
# Y15 경찰서별 경찰관 현황 전처리 노트북


## 전처리 규칙 
1. 컬럼명 표준화: `시도`, `시군구`, `경찰서명`, `부서`, `주소`, `전화번호`, `위도`, `경도`, `X좌표`, `Y좌표`, `정원`, `현원`, `총원`, `남자`, `여자` 등
2. 중복 제거, 문자열 공백/전각공백 정리, 결측치 처리(`문자형 → '미상'`, `숫자형 → 중앙값`)
3. 인원/합계형 컬럼 숫자형 변환(천단위 콤마 제거 등)
4. 명칭/접미 통일: `경찰서명` → '…경찰서' 접미 보정
5. 파생지표: `충원율(%)`, `여경비율(%)`, `남경비율(%)` (존재할 때만 생성)
6. 컬럼 순서: **식별(행정구역/기관) → 연락처/좌표 → 수치지표 → 기타**
7. 좌표/숫자 포맷: 위/경도 소수점 6자리, TM 좌표 소수점 2자리


In [ ]:

# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import re
from pathlib import Path
from IPython.display import display

# 입력/출력 경로 설정
SRC = Path("/mnt/data/Y15_경찰청_전국 경찰서별 경찰관 현황_20231231.csv")
OUT_CSV = Path("/mnt/data/Y15_경찰청_전국경찰서별_경찰관현황_20231231_전처리.csv")
os.makedirs(output_dir, exist_ok=True)  # 폴더가 없으면 자동 생성 

print("입력 파일 존재 여부:", SRC.exists(), "\n경로:", SRC)
print("출력 파일 경로:", OUT_CSV)


In [ ]:

def normalize_col(col: str) -> str:
    """ 컬럼명 표준화 """
    col = str(col).strip()
    col = re.sub(r"\s+", "", col)  # 공백류 제거
    mapping = {
        "시도":"시도","시도명":"시도","광역시도":"시도",
        "시군구":"시군구","시군구명":"시군구","구군":"시군구",
        "경찰서":"경찰서명","경찰서명":"경찰서명","관할서":"경찰서명",
        "부서":"부서","부서명":"부서",
        "주소":"주소","도로명주소":"주소",
        "연락처":"전화번호","전화번호":"전화번호",
        "위도":"위도","경도":"경도","x좌표":"X좌표","y좌표":"Y좌표",
        "총원":"총원","정원":"정원","현원":"현원",
        "남자":"남자","여자":"여자"
    }
    return mapping.get(col, col)

def tidy_strings(series: pd.Series) -> pd.Series:
    """문자열 컬럼의 전각공백/nbps/양끝공백 정리"""
    return (
        series.astype(str)
        .str.replace("\u3000", " ", regex=False)
        .str.replace("\xa0", " ", regex=False)
        .str.strip()
    )

def to_numeric_safe(series: pd.Series) -> pd.Series:
    """천단위 콤마/공백 제거 후 숫자형 변환"""
    return pd.to_numeric(
        series.astype(str).str.replace(",", "", regex=False).str.strip(),
        errors="coerce"
    )


In [ ]:

# 인코딩 자동 판별 시도
encodings = ["utf-8-sig", "cp949", "utf-8"]
df = None
for enc in encodings:
    try:
        df = pd.read_csv(SRC, encoding=enc)
        print(f"✅ 인코딩 성공: {enc}")
        break
    except Exception as e:
        print(f"⚠️ 인코딩 실패: {enc} -> {e}")
if df is None:
    raise RuntimeError("입력 CSV를 읽지 못했습니다. 인코딩을 확인하세요.")

print("원본 컬럼:", list(df.columns))


In [ ]:

# 1) 컬럼명 표준화
df.columns = [normalize_col(c) for c in df.columns]

# 2) 문자열 컬럼 클린업
for c in df.columns:
    if df[c].dtype == "object":
        df[c] = tidy_strings(df[c])

# 3) 완전 중복 행 제거
before = len(df)
df = df.drop_duplicates()
after = len(df)
print(f"중복 제거: {before - after}행 제거, 현재 {after}행")

# 4) 결측치 처리: 문자 → '미상', 숫자 → 중앙값
for c in df.columns:
    if df[c].dtype == "object":
        df[c] = df[c].replace({"": np.nan}).fillna("미상")
    else:
        med = df[c].median() if df[c].notna().any() else 0
        df[c] = df[c].fillna(med)

print("정리된 컬럼:", list(df.columns))


In [ ]:

# 인원/합계 관련 컬럼 숫자형 변환
numeric_like_cols = [c for c in df.columns if re.search(r"(총원|정원|현원|남자|여자|인원|계|명)$", c)]
for c in numeric_like_cols:
    df[c] = to_numeric_safe(df[c])

print("숫자형 변환 대상:", numeric_like_cols)


In [ ]:

# '경찰서명' 접미 통일 (…경찰서)
if "경찰서명" in df.columns:
    df["경찰서명"] = tidy_strings(df["경찰서명"].astype(str))
    df["경찰서명"] = df["경찰서명"].str.replace(r"(경찰서)?$", "경찰서", regex=True)
    df["경찰서명"] = df["경찰서명"].str.replace("경찰서경찰서", "경찰서", regex=False)

# 시도/시군구 공백 제거
for k in ["시도", "시군구"]:
    if k in df.columns:
        df[k] = df[k].str.replace(r"\s+", "", regex=True)

# 파생지표: 충원율, 남/여 비율 (존재할 때만 생성)
if set(["현원","정원"]).issubset(df.columns):
    df["충원율(%)"] = np.where(df["정원"]>0, (df["현원"]/df["정원"]*100).round(2), np.nan)
if set(["남자","여자","현원"]).issubset(df.columns):
    df["여경비율(%)"] = np.where(df["현원"]>0, (df["여자"]/df["현원"]*100).round(2), np.nan)
    df["남경비율(%)"] = np.where(df["현원"]>0, (df["남자"]/df["현원"]*100).round(2), np.nan)

df.head(3)


In [ ]:

# 컬럼 순서
preferred_order = [c for c in ["시도","시군구","경찰서명","부서","주소"] if c in df.columns] +                   [c for c in ["전화번호"] if c in df.columns] +                   [c for c in ["위도","경도","X좌표","Y좌표"] if c in df.columns] +                   [c for c in ["정원","현원","총원","남자","여자","충원율(%)","여경비율(%)","남경비율(%)"] if c in df.columns]

remaining = [c for c in df.columns if c not in preferred_order]
df = df[preferred_order + remaining]

# 좌표 자릿수 포맷
for col, dec in [("위도",6),("경도",6),("X좌표",2),("Y좌표",2)]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").round(dec)

df.head(5)


In [ ]:

# 저장
df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print("💾 저장 완료 ->", OUT_CSV)

# 미리보기
display(df.head(20))
print("컬럼:", list(df.columns))
print("shape:", df.shape)
